In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import asyncio
import time
from openai import AsyncOpenAI  # 注意这里导入AsyncOpenAI
from typing import List, Dict

# config.py
from dotenv import load_dotenv # type: ignore

# 加载 .env 文件
load_dotenv()

# 读取
ZetaTechs_api_key = os.getenv('ZETATECHS_API_KEY')
ZetaTechs_api_base = os.getenv('ZETATECHS_API_BASE')

In [3]:
class AsyncAPIClient:
    def __init__(self, max_concurrency=100):
        self.client = AsyncOpenAI(api_key=ZetaTechs_api_key, base_url=ZetaTechs_api_base) # AsyncOpenAI 专门用于异步
        # 限制并发请求数，避免触发API限制
        self.semaphore = asyncio.Semaphore(max_concurrency)  # 同时最多5个请求
        
    async def generate_response(self, prompt: str) -> str:
        """发送单个请求到OpenAI API"""
        try:
            # 使用信号量控制并发
            async with self.semaphore: # 确保同时只有五个请求在处理
                response = await self.client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": """你是一个专业的系统提示词工程师。你的任务是根据给定的对话内容，生成恰当的系统提示词。

请注意以下要求：
1. 系统提示词应该定义AI助手的角色、行为准则和专业领域
2. 提示词应该包含AI助手的性格特征和交互风格
3. 明确指出AI助手在回答时应遵循的原则和限制
4. 确保提示词简洁专业，通常以"你是..."或"作为..."等开头
5. 避免生成看起来像用户指令的提示词

示例格式：
"你是一个专业的[领域]专家，擅长[特定能力]。在回答问题时，你应该[行为准则]，并始终保持[性格特征]。"

请分析用户提供的对话内容，生成一个符合以上要求的系统提示词。直接返回提示词内容，不需要任何解释和额外的内容。"""},
                        {"role": "user", "content": prompt}
                    ]
                )
                return response.choices[0].message.content
        except Exception as e:
            print(f"请求失败: {e}")
            return f"Error: {str(e)}"

    async def batch_generate(self, prompts: List[str]) -> List[str]:
        """批量处理多个提示词"""
        # 创建所有任务
        tasks = [self.generate_response(prompt) for prompt in prompts]
        # 并发执行所有任务
        results = await asyncio.gather(*tasks)
        return results

In [4]:
import json

# 定义文件路径
train_file_path = 'base_stage_data_raw/train.json'
valid_file_path = 'base_stage_data_raw/valid.json'

# 读取文件
with open(train_file_path, 'r', encoding='utf-8') as file:
    train_data = json.load(file)

with open(valid_file_path, 'r', encoding='utf-8') as file:
    valid_data = json.load(file)

# 提取 text 字段
train_data_texts = [item['text'] for item in train_data if 'text' in item]
valid_data_texts = [item['text'] for item in valid_data if 'text' in item]

# 截取前n个 - 训练数据8万，验证数据2万
train_data_texts = train_data_texts[:10000]
valid_data_texts = valid_data_texts[:500]


In [5]:
# 为Jupyter环境特别优化的运行方式
async def run_async_tasks():
    max_concurrency = 100
    client = AsyncAPIClient(max_concurrency)
    
    # test_prompts = [
    #     "讲个笑话",
    #     "介绍下Python",
    #     "解释什么是异步编程",
    #     "写一首诗",
    #     "讲解下OpenAI API"
    # ]
    
    results = await client.batch_generate(train_data_texts)
    
    # for i, response in enumerate(results, 1):
    #     print(f"\n--- 响应 {i} ---")
    #     print(response)
    return results

In [6]:
# 在Jupyter中运行这个
generate_results = await run_async_tasks()

请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection error.
请求失败: Connection

In [7]:
import json

# 假设这是我们要保存的列表
# data = [
#     {"name": "Alice", "age": 30},
#     {"name": "Bob", "age": 25},
#     {"name": "Charlie", "age": 35}
# ]

# 定义保存的文件路径
file_path = "base_stage_data_raw/generate_results_test.json"

# 将列表保存为 JSON 文件
with open(file_path, "w", encoding="utf-8") as file:
    json.dump(generate_results, file, ensure_ascii=False, indent=4)

print(f"列表已成功保存为 JSON 文件: {file_path}")

列表已成功保存为 JSON 文件: base_stage_data_raw/generate_results_test.json
